A simple Travelling Salesman Problem to solve (optimize) with IBM Cplex for learning purposes

_"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"_

*(Wikipedia.org, 2023)* - https://en.wikipedia.org/wiki/Travelling_salesman_problem

![problem-studied](./documentation/problem.png)

# BEFORE CODING: INSTALL CPLEX AND SETUP YOUR PYTHON ENVIRONEMENT
0. Make sure you have Python **3.7** or **3.8** installed (_Cplex currently do not support newer versions!_)
1. Download Cplex from **IBM official website** and install it: https://www.ibm.com/products/ilog-cplex-optimization-studio/cplex-optimizer
2. Go to the `CPLEX_HOME/cplex/python/3.8/x86-64_osx/cplex/` (_choose the right python version and OS platform here!_) sub-folder inside the Cplex install folder (`/Applications/CPLEX_Studio210` in MacOS) and run: `python setup.py install --home PATH_TO_PYTHON_PACKAGES/cplex` or `python3.8 setup.py install --home PATH_TO_PYTHON_PACKAGES/cplex` if you have several versions of python installed

--> _for example:_ **PATH_TO_PYTHON_PACKAGES**=`/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages`

3. **Alternative:** Install cplex using `pip install cplex dcplex` or `pip3.8 install cplex dcplex` if you have several versions of cplex installed

In [22]:
#0. IMPORT ALL USED LIBRARIES
from random import randint
import numpy as np
from docplex.mp.model import Model
import cplex

In [12]:
# I. BUILD A RANDOM TSP INSTANCE 
def build_random_instance(nbr_cities=6, min_distance=1, max_distance=15):
    start = randint(0, nbr_cities-1) # 1. random generate the starting city
    distances = np.random.uniform(min_distance, max_distance, (nbr_cities, nbr_cities)) # 2. random generate a matrix of paths between cities
    distances = np.triu(distances, k=1)
    distances += distances.T # 3. the distance between city a -> city b should be equal to the between city b -> city a
    np.fill_diagonal(distances, 0) # 4. replace all distance for a city to itself (the diagonal) to 0 
    return distances, start

distances, start = build_random_instance(5,1,15)
print(distances)
print(start)

[[ 0.          2.33543911  8.79022696 10.92399927  4.20015584]
 [ 2.33543911  0.          8.13175513  7.72822447 14.9250001 ]
 [ 8.79022696  8.13175513  0.          6.6774187   8.93846642]
 [10.92399927  7.72822447  6.6774187   0.         10.03828901]
 [ 4.20015584 14.9250001   8.93846642 10.03828901  0.        ]]
2


In [24]:
#II. BUILD THE CPLEX MODEL: variables, constraints, and objective function
m = Model(name='TSP')

AttributeError: module 'cplex' has no attribute 'Cplex'

![calcul-of-subsets](./documentation/calcul.png)